In [4]:
import pandas as pd
# Load the uploaded CSV files
acb_data = pd.read_csv("D:\\Github anyud\\final\\Data_stock\\ACB Historical Data.csv")
aapl_data = pd.read_csv('D:\\Github anyud\\final\\Data_stock\\AAPL Historical Data.csv')
bid_data = pd.read_csv('D:\\Github anyud\\final\\Data_stock\\BID Historical Data.csv')
fpt_data = pd.read_csv('D:\\Github anyud\\final\\Data_stock\\FPT Historical Data.csv')
googl_data = pd.read_csv('D:\\Github anyud\\final\\Data_stock\\GOOGL Historical Data.csv')

# Display the first few rows of each dataframe to understand the structure and columns
acb_data.head()


,Date,Price,Open,High,Low,Vol.,Change %
0,09/11/2024,7.630,7.720,7.760,7.460,320.28K,-0.52%
1,09/10/2024,7.670,7.810,7.810,7.550,265.39K,-2.29%
2,09/09/2024,7.850,7.520,7.950,7.520,443.07K,7.09%
3,09/06/2024,7.330,7.560,7.640,7.170,390.09K,-3.68%
4,09/05/2024,7.610,7.730,7.845,7.540,552.62K,-0.65%


In [11]:
# Function to preprocess the data with updated handling for 'Change %' column
def preprocess_data(df, stock_id):
    df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['Weekday'] = df['Date'].dt.day_name()
    
    df['Stock_ID'] = stock_id
    df['Vol.'] = df['Vol.'].apply(lambda x: float(str(x).replace('M', '').replace('K', '')) * 1_000_000 if 'M' in str(x) else float(str(x).replace('K', '')) * 1_000 if 'K' in str(x) else float(x))
    df['Change %'] = df['Change %'].astype(str).str.replace('%', '').astype(float)
    
    return df

# Re-process the data with the updated function
acb_data_processed = preprocess_data(acb_data, 1)
aapl_data_processed = preprocess_data(aapl_data, 2)
bid_data_processed = preprocess_data(bid_data, 3)
fpt_data_processed = preprocess_data(fpt_data, 4)
googl_data_processed = preprocess_data(googl_data, 5)


# Select relevant columns for each table
date_dimension_columns = ['Date', 'Year', 'Month', 'Day', 'Weekday']
stock_trading_facts_columns = ['Date', 'Stock_ID', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']

# Prepare Date_Dimension table data
date_dimension_data = pd.concat([acb_data_processed[date_dimension_columns], aapl_data_processed[date_dimension_columns], bid_data_processed[date_dimension_columns], fpt_data_processed[date_dimension_columns]]).drop_duplicates().reset_index(drop=True)

# Prepare Stock_Trading_Facts table data
stock_trading_facts_data = pd.concat([acb_data_processed[stock_trading_facts_columns], bid_data_processed[stock_trading_facts_columns], fpt_data_processed[stock_trading_facts_columns]]).reset_index(drop=True)
exception_fact = pd.concat([aapl_data_processed[stock_trading_facts_columns], googl_data_processed[stock_trading_facts_columns]]) 
# Prepare Stock_Dimension table data
stock_dimension_data = pd.DataFrame({
    'Stock_ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Stock_Name': ['ACB', 'AAPL', 'BID', 'FPT', 'GOOGL']
})

# Display the first few rows of each table data
date_dimension_data.head()



ValueError: could not convert string to float: '1,899.50'

In [8]:
stock_trading_facts_data.head()

NameError: name 'stock_trading_facts_data' is not defined

In [104]:
stock_dimension_data.head()

,Stock_ID,Stock_Name
0,1,AAA
1,2,AAPL
2,3,ACB
3,4,BID
4,5,CTG


In [9]:
# Remove commas and convert to float64
stock_trading_facts_data['Price'] = stock_trading_facts_data['Price'].str.replace(',', '').astype(float)
stock_trading_facts_data['Open'] = stock_trading_facts_data['Open'].str.replace(',', '').astype(float)
stock_trading_facts_data['High'] = stock_trading_facts_data['High'].str.replace(',', '').astype(float)
stock_trading_facts_data['Low'] = stock_trading_facts_data['Low'].str.replace(',', '').astype(float)

exception_fact.dtypes

# exception_fact['Price'] = exception_fact['Price'].astype(float)
# exception_fact['Open'] = exception_fact['Open'].astype(float)
# exception_fact['High'] = exception_fact['High'].astype(float)
# exception_fact['Low'] = exception_fact['Low'].astype(float)
# Check the data types again
stock_trading_facts_data.dtypes
# stock_trading_facts_data.to_csv('test.csv', index=False)

NameError: name 'stock_trading_facts_data' is not defined

In [108]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

# Thông tin kết nối cơ sở dữ liệu
db_user = 'postgres'
db_password = '123'
db_host = 'localhost'
db_port = '5432'
db_name = 'thesis_stock'

# Tạo chuỗi kết nối
connection_string = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

# Lệnh SQL để tạo schema
create_schema_sql = "CREATE SCHEMA IF NOT EXISTS stock;"

# Kết nối tới cơ sở dữ liệu PostgreSQL và tạo schema nếu chưa tồn tại
try:
    connection = psycopg2.connect(connection_string)
    cursor = connection.cursor()
    cursor.execute(create_schema_sql)
    connection.commit()
    print("Schema đã được tạo thành công")
except Exception as e:
    print(f"Lỗi khi kết nối hoặc thực thi lệnh SQL: {e}")
finally:
    if connection:
        cursor.close()
        connection.close()
        print("Đã đóng kết nối đến cơ sở dữ liệu PostgreSQL")

# Đổi tên cột để chắc chắn khớp với tên mong muốn
stock_trading_facts_data.rename(columns={
    'Vol.': 'Volume',
    'Change %': 'Change_Percentage'
}, inplace=True)

exception_fact.rename(columns={
    'Vol.': 'Volume',
    'Change %': 'Change_Percentage'
}, inplace=True)

# Xử lý các giá trị không xác định (NA hoặc inf) cho DataFrame stock_trading_facts_data
stock_trading_facts_data.fillna({'Volume': 0}, inplace=True)
stock_trading_facts_data.replace({'Volume': {float('inf'): 0, -float('inf'): 0}}, inplace=True)

# Xử lý các giá trị không xác định (NA hoặc inf) cho DataFrame exception_fact
exception_fact.fillna({'Volume': 0}, inplace=True)
exception_fact.replace({'Volume': {float('inf'): 0, -float('inf'): 0}}, inplace=True)

# Đảm bảo rằng các cột có kiểu dữ liệu chính xác
Date_Dimension_data = date_dimension_data.astype({
    'Date': 'datetime64[ns]',
    'Year': 'int',
    'Month': 'int',
    'Day': 'int',
    'Weekday': 'object'
})

Stock_Dimension_data = stock_dimension_data.astype({
    'Stock_ID': 'int',
    'Stock_Name': 'object'
})

Stock_Trading_Facts_data = stock_trading_facts_data.astype({
    'Date': 'datetime64[ns]',
    'Stock_ID': 'int',
    'Price': 'float',
    'Open': 'float',
    'High': 'float',
    'Low': 'float',
    'Volume': 'int',
    'Change_Percentage': 'float'
})

Exception_Stock_Trading_Facts_data = exception_fact.astype({
    'Date': 'datetime64[ns]',
    'Stock_ID': 'int',
    'Price': 'float',
    'Open': 'float',
    'High': 'float',
    'Low': 'float',
    'Volume': 'int',
    'Change_Percentage': 'float'
})

# Tạo kết nối với SQLAlchemy
engine = create_engine(connection_string)

# Chèn dữ liệu vào các bảng trong schema 'stock'
Date_Dimension_data.to_sql('Dim_Date', engine, schema='stock', if_exists='append', index=False)
Stock_Dimension_data.to_sql('Dim_Stock', engine, schema='stock', if_exists='append', index=False)
Stock_Trading_Facts_data.to_sql('Fact_Stock', engine, schema='stock', if_exists='append', index=False)
Exception_Stock_Trading_Facts_data.to_sql('Fact_Stock', engine, schema='stock', if_exists='append', index=False)

print("Dữ liệu đã được chèn vào cơ sở dữ liệu thành công")


Schema đã được tạo thành công
Đã đóng kết nối đến cơ sở dữ liệu PostgreSQL
Dữ liệu đã được chèn vào cơ sở dữ liệu thành công
